In [22]:
import pyautogui
import shutil
import time
from PIL import ImageGrab
from PIL import Image
import os
import keyboard
import pyperclip
import wx
import lxk

image_path = 'D:/AutoClickPicture/'
try:
    os.mkdir(image_path)
except:
    pass
def click_image(image, confidence=0.9, lag_x=0,lag_y=0,wait=0, drag=None, double=False):
    
    time.sleep(wait)

    # 获取屏幕分辨率
    screen_width, screen_height = pyautogui.size()

    # 全屏截图
    screen_image = ImageGrab.grab()

    # 在屏幕截图中查找匹配
    location = pyautogui.locate(image, screen_image, confidence=confidence)
    if location:
        # 匹配成功，返回位置信息
        x, y, a, b = location
        if drag==None:
            if double==False:
                pyautogui.click(x+a//2+lag_x, y+b//2+lag_y)
            else:
                pyautogui.doubleClick(x+a//2+lag_x, y+b//2+lag_y)
        else:
            pyautogui.moveTo(x+a//2+lag_x, y+b//2+lag_y)
            pyautogui.mouseDown()
            pyautogui.moveRel(drag[0],drag[1],0.2)
            pyautogui.mouseUp()
        time.sleep(0.2)
        return 1
    else:
        # 匹配失败，返回None
        return 0

In [ ]:
class btnFrame(wx.Frame):
    def __init__(self, parent, btn_group):        
        self.flag = -1
        self.pic_group =[]
        self.image_path = image_path+'temp/'
        h =150
        w =len(btn_group)*30+100
        wx.Frame.__init__(self, parent, title='My Frame', size=(h, w))
        
        
        sizer = wx.BoxSizer(wx.VERTICAL)
        for i in btn_group:
            exec(f"self.{i}=wx.Button(self, label='{btn_group[i]}')") # 创建btn
            exec(f"sizer.Add(self.{i}, 0, wx.ALL, 5)") # 添加进
            exec(f"self.{i}.Bind(wx.EVT_BUTTON, self.On_{i})")

        self.SetSizer(sizer)
        
        # 初始化窗口
        self.SetPosition(wx.Point(wx.DisplaySize()[0] - h, wx.DisplaySize()[1] - w))
        self.Show()
        self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE | wx.STAY_ON_TOP)
        
    def On_toggle_btn(self, event):
        if self.GetWindowStyle() & wx.STAY_ON_TOP:
            self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE)
        else:
            self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE | wx.STAY_ON_TOP)
            
    def On_add_btn(self, event):
        self.flag=+1
        self.pic_group.insert(self.flag,ImageGrab.grabclipboard())
        
    def On_try_btn(self, event):
        click_image(self.pic_group[self.flag])
        
    def On_delete_btn(self, event):
        self.flag=-1
        self.pic_group.pop(self.flag)
    
    def On_debug_btn(self, event):
        self.flag=-1
        self.pic_group.pop(self.flag)
    
    def On_save_btn(self, event):
        try:
            os.mkdir(self.image_path)
        except:
            shutil.rmtree(self.image_path)
            os.mkdir(self.image_path)
        for i in range(0,len(self.pic_group)):
            self.pic_group[i].save(self.image_path+f'{i}.png')
        
    def On_load_btn(self, event):
        self.pic_group=[]
        self.flag=-1
        for i in range(0,len(os.listdir(self.image_path))):
            image = Image.open(self.image_path+ f'{i}.png')
            self.pic_group.append(image)
            
    def On_start_btn(self, event):
        for i in range(0,len(self.pic_group)):
            while click_image(self.pic_group[i],wait=0.2)==0:
                time.sleep(0.2)
        
try:
    del app
except:
    pass
app = wx.App()
btn_group = {'toggle_btn':'始终置顶','add_btn':'添加','try_btn':'尝试点击','delete_btn':'删除',\
                     'debug_btn':'调试','save_btn':'保存','load_btn':'加载','start_btn':'执行'}
frame = btnFrame(None,btn_group)
app.MainLoop()

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

In [19]:
while True:
    keyboard.wait('ctrl')
    pyautogui.hotkey('ctrl','c')
    x=pyperclip.paste()
    y=f'''drop table if exists {x}_old;
rename table {x} to {x}_old ;
rename table {x}_new to {x};'''
    pyautogui.hotkey('ctrl','a')
    pyperclip.copy(y)
    pyautogui.hotkey('ctrl','v')

KeyboardInterrupt: 

In [28]:
import re
keyboard.wait('f2')
while True:
    
    x = 0
    for i in range(0,len(os.listdir(image_path))):
        image = Image.open(image_path+ f'{i}.png')
        if i==0:
            click_image(image,double=True)
            continue
        click_image(image,wait=0.2)
        if i==1:
            pyautogui.hotkey('ctrl','a')
            pyautogui.hotkey('ctrl','c')
            x=pyperclip.paste()
            y=re.sub('清空','重命名',x)
            x=re.sub('清空','',x)
            pyperclip.copy(y)
            pyautogui.hotkey('ctrl','v')
        if i ==2:
            pyautogui.hotkey('ctrl','a')
            y=f'''drop table if exists {x}_old;
rename table {x} to {x}_old ;
rename table {x}_new to {x};'''
            pyperclip.copy(y)
            pyautogui.hotkey('ctrl','v')

KeyboardInterrupt: 